In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import molsysmt as msm
import openexplorer as oe
import numpy as np
from simtk import unit
from simtk.openmm import app

In [ ]:
modeller = msm.convert('alanine_tetrapeptide.pdb', to_form='openmm.Modeller')

topology = modeller.topology
positions = modeller.positions

forcefield = app.ForceField('amber10.xml', 'amber10_obc.xml')
system = forcefield.createSystem(topology, nonbondedMethod=app.NoCutoff, #constraints=app.HBonds,
                                implicitSolvent=app.OBC2, soluteDielectric=1.0, solventDielectric=78.5)

In [ ]:
explorer = oe.Explorer(topology, system, platform='CUDA')
explorer.set_coordinates(positions)
explorer.md.langevin(1000)
explorer.quench.l_bfgs()

In [ ]:
temperature = 500.0*unit.kelvin
timestep = 2.0*unit.femtoseconds
collision_rate = 1.0/unit.picoseconds # Openmm suggests 1 ps-1 but will it be the same for implicit solvent? 64?
explorer.md.langevin.set_parameters(temperature=temperature, timestep=timestep,
                                    collision_rate=collision_rate)

threshold = 0.01*unit.angstroms

time_langevin_cicle = 1.0*unit.picoseconds
steps_langevin_cicle = int(time_langevin_cicle/timestep)
n_attempts_per_minimum = 150

minima_coordinates = []
minima_potential_energy = []
n_minima = 0

explorer.quench.l_bfgs()
minima_coordinates.append(explorer.get_coordinates())
minima_potential_energy.append(explorer.get_potential_energy())
n_minima += 1


current_minimum_index = 0

while current_minimum_index < n_minima:
    
    
    current_minimum_coordinates = minima_coordinates[current_minimum_index]
    n_attempts = 0
    n_cicles = 0
    
    coordinates = current_minimum_coordinates
    explorer.set_coordinates(coordinates)
    explorer.set_velocities_to_temperature(temperature)
    
    while n_attempts<n_attempts_per_minimum:
    
        pre_coordinates=explorer.get_coordinates()
        
        explorer.md.langevin(steps_langevin_cicle)
        coordinates = explorer.get_coordinates()
        velocities = explorer.get_velocities()
        
        explorer.quench.l_bfgs()
    
        if explorer.distance.least_rmsd(current_minimum_coordinates) <= threshold:
            explorer.set_coordinates(coordinates)
            explorer.set_velocities(velocities)

        else:
        
            n_attempts+=1
        
            new_minimum=True
            for minimum_index in range(n_minima):
                if minimum_index!=current_minimum_index:
                    if explorer.distance.least_rmsd(minima_coordinates[minimum_index]) <= threshold:
                        new_minimum = False
                        break
            if new_minimum:
                minima_coordinates.append(explorer.get_coordinates())
                minima_potential_energy.append(explorer.get_potential_energy())
                n_minima+=1
        
            explorer.set_coordinates(pre_coordinates)
            explorer.set_velocities_to_temperature(temperature)

            print(current_minimum_index, n_attempts, n_cicles, n_minima)
        
        n_cicles+=1
    
    current_minimum_index+=1

Podemos calcular la distancia entre configuraciones para hacer un mapa por proximidad: mds o red con threshold.